In [14]:
from selenium import webdriver
import webbrowser
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent
import time

import os

import pandas as pd
from bs4 import BeautifulSoup
import requests

from webdriver_manager.chrome import ChromeDriverManager


In [15]:
#replace path
data = pd.read_csv(str(os.getcwd()).replace('code', '')+'/data/cleaned_prof_data_v4.csv')


#clean up data
data = data.sort_values('name')
data = data.reset_index(drop = True)
data

,name,affiliation,homepage,scholar_id,citations,h-index
0,A Min Tjoa,TU Wien,http://www.ifs.tuwien.ac.at/tjoa,x8qCMhcAAAAJ,6069.0,40.0
1,A. Akbari Azirani,IUST,http://ce.iust.ac.ir/page.php?slct_pg_id=6537&...,pCil4_cAAAAJ,1326.0,18.0
2,A. Aldo Faisal,Imperial College London,https://www.imperial.ac.uk/people/a.faisal,WjHjbrwAAAAJ,6780.0,31.0
3,A. Antony Franklin,IIT Hyderabad,http://www.iith.ac.in/~antony/index.html,LVfqLuoAAAAJ,NaN,NaN
4,A. B. M. Alim Al Islam,BUET,https://sites.google.com/site/abmalimalislam,K-AIPzQAAAAJ,562.0,11.0
...,...,...,...,...,...,...
20644,Ítalo Cunha,UFMG,https://www.dcc.ufmg.br/~cunha,bTAky1EAAAAJ,2036.0,23.0
20645,Óscar Cánovas,University of Murcia,http://webs.um.es/ocanovas/miwiki/doku.php,iAPkCb8AAAAJ,625.0,13.0
20646,Özgür Erkent,Hacettepe University,https://web.cs.hacettepe.edu.tr/~ozgurerkent,5QMAbisAAAAJ,348.0,11.0
20647,Özgür Ulusoy,Bilkent University,http://www.cs.bilkent.edu.tr/~oulusoy,M_vU9pQAAAAJ,4716.0,37.0


In [5]:
data.head(10)

,name,affiliation,homepage,scholar_id,citations,h-index
0,A Min Tjoa,TU Wien,http://www.ifs.tuwien.ac.at/tjoa,x8qCMhcAAAAJ,6069.0,40.0
1,A. Akbari Azirani,IUST,http://ce.iust.ac.ir/page.php?slct_pg_id=6537&...,pCil4_cAAAAJ,1326.0,18.0
2,A. Aldo Faisal,Imperial College London,https://www.imperial.ac.uk/people/a.faisal,WjHjbrwAAAAJ,6780.0,31.0
3,A. Antony Franklin,IIT Hyderabad,http://www.iith.ac.in/~antony/index.html,LVfqLuoAAAAJ,NaN,NaN
4,A. B. M. Alim Al Islam,BUET,https://sites.google.com/site/abmalimalislam,K-AIPzQAAAAJ,562.0,11.0
5,A. B. Siddique,University of Kentucky,http://cs.uky.edu/~siddique,pVnchfsAAAAJ,78.0,6.0
6,A. Bhaskar,BITS Pilani-Goa,https://www.bits-pilani.ac.in/goa/abaskar/profile,WNrQSakAAAAJ,NaN,NaN
7,A. C. Cem Say,Boğaziçi University,https://www.cmpe.boun.edu.tr/~say,rOum2XsAAAAJ,835.0,15.0
8,A. C. W. Finkelstein,City University of London,https://finkelstein.uk,n8xuCVkAAAAJ,16019.0,62.0
9,A. Cüneyd Tantug,Istanbul Technical University,http://tantug.com,TTawdWMAAAAJ,517.0,13.0


In [18]:
# Make sure you allow remote automation in "develop" tab
#will create a new window, if not done running and popup comes up press "continue"
driver = webdriver.Chrome('/Users/kevins/Documents/CSImpact/code/chromedriver')


/var/folders/sw/rc481v691q5883ld6bm6jzq80000gn/T/ipykernel_3827/1616792208.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/kevins/Documents/CSImpact/code/chromedriver')


In [7]:
earliest_paper_years = []
total_rows = data.shape[0]
#test_data = data.head(100)
#total_test_rows = test_data.shape[0]

for i in range(total_rows):
    s_id = data.iloc[i,3]
    if not pd.isna(data.iloc[i,4]):
        html_text = requests.get(f'https://scholar.google.com/citations?hl=en&user={s_id}&view_op=list_works&sortby=pubdate').text
        soup = BeautifulSoup(html_text, 'lxml')
        values = soup.find_all('button', id = 'gsc_bpf_more')
        
        
        options = Options()
        ua = UserAgent()
        userAgent = ua.random
        options.add_argument(f'user-agent={userAgent}')
        driver = webdriver.Chrome(chrome_options=options, executable_path='./chromedriver')

        driver.get(f'https://scholar.google.com/citations?hl=en&user={s_id}&view_op=list_works&sortby=pubdate')
        text1 = driver.page_source
        while True:
            try:
                button = driver.find_element_by_id('gsc_bpf_more')
                button.click()
            except ElementNotInteractableException:
                final_text = driver.page_source
                break
            except NoSuchElementException:
                driver.get(f'https://scholar.google.com/citations?hl=en&user={s_id}&view_op=list_works&sortby=pubdate')
                break
            text2=driver.page_source
            if text1 == text2:
                final_text = text2
                break
            else:
                text1 = text2

        final_soup = BeautifulSoup(final_text, 'lxml')
        years = final_soup.find_all('td', class_ = 'gsc_a_y')
        years_text = []
        for year in years:
            years_text.append(year.text)

        clean_years = [year_text for year_text in years_text if year_text != '']

        min_y = int(clean_years[0])
        for clean_y in clean_years:
            if int(clean_y) < min_y:
                min_y = int(clean_y)
        earliest_paper_years.append(str(min_y))

    else:
        earliest_paper_years.append('')

Error occurred during loading data. Trying to use cache server https://fake-useragent.herokuapp.com/browsers/0.1.11
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/site-packages/fake_useragent/utils.py", line 154, in load
    for item in get_browsers(verify_ssl=verify_ssl):
  File "/usr/local/lib/python3.9/site-packages/fake_useragent/utils.py", line 99, in get_browsers
    html = html.split('<table class="w3-table-all notranslate">')[1]
IndexError: list index out of range
/var/folders/sw/rc481v691q5883ld6bm6jzq80000gn/T/ipykernel_3827/2679647184.py:18: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_options=options, executable_path='./chromedriver')
/var/folders/sw/rc481v691q5883ld6bm6jzq80000gn/T/ipykernel_3827/2679647184.py:18: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=options, executable_path='./chromedriver')
/var/folders/sw/rc4

WebDriverException: Message: unknown error: Chrome failed to start: exited abnormally.
  (unknown error: DevToolsActivePort file doesn't exist)
  (The process started from chrome location /Applications/Google Chrome.app/Contents/MacOS/Google Chrome is no longer running, so ChromeDriver is assuming that Chrome has crashed.)
Stacktrace:
0   chromedriver                        0x000000010069a079 chromedriver + 4444281
1   chromedriver                        0x0000000100626403 chromedriver + 3970051
2   chromedriver                        0x00000001002c1038 chromedriver + 409656
3   chromedriver                        0x00000001002e4e4c chromedriver + 556620
4   chromedriver                        0x00000001002e0836 chromedriver + 538678
5   chromedriver                        0x000000010031a44a chromedriver + 775242
6   chromedriver                        0x0000000100314683 chromedriver + 751235
7   chromedriver                        0x00000001002eaa45 chromedriver + 580165
8   chromedri

In [81]:
data['h-index'].max()

219.0

In [82]:
data['citations'].max()

532373.0

In [6]:
len(earliest_paper_years)

744

In [7]:
data

,name,affiliation,homepage,scholar_id,citations,h-index
0,A Min Tjoa,TU Wien,http://www.ifs.tuwien.ac.at/tjoa,x8qCMhcAAAAJ,6069.0,40.0
1,A. Akbari Azirani,IUST,http://ce.iust.ac.ir/page.php?slct_pg_id=6537&...,pCil4_cAAAAJ,1326.0,18.0
2,A. Aldo Faisal,Imperial College London,https://www.imperial.ac.uk/people/a.faisal,WjHjbrwAAAAJ,6780.0,31.0
3,A. Antony Franklin,IIT Hyderabad,http://www.iith.ac.in/~antony/index.html,LVfqLuoAAAAJ,NaN,NaN
4,A. B. M. Alim Al Islam,BUET,https://sites.google.com/site/abmalimalislam,K-AIPzQAAAAJ,562.0,11.0
...,...,...,...,...,...,...
20644,Ítalo Cunha,UFMG,https://www.dcc.ufmg.br/~cunha,bTAky1EAAAAJ,2036.0,23.0
20645,Óscar Cánovas,University of Murcia,http://webs.um.es/ocanovas/miwiki/doku.php,iAPkCb8AAAAJ,625.0,13.0
20646,Özgür Erkent,Hacettepe University,https://web.cs.hacettepe.edu.tr/~ozgurerkent,5QMAbisAAAAJ,348.0,11.0
20647,Özgür Ulusoy,Bilkent University,http://www.cs.bilkent.edu.tr/~oulusoy,M_vU9pQAAAAJ,4716.0,37.0
